<a href="https://www.kaggle.com/code/edrickkesuma/explanatory-votingclassifier-power-averaging?scriptVersionId=91204766" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# VotingClassifier + Power Averaging - TPS Oct 2021

## What is Power Averaging?

The main idea behind Power Averaging is that we want **highly correlated** submissions to combine to get a better AUC.

But why does Power Averaging work?
1. AUC judges score based on ranking only. Example: (0,1,2) has the same AUC as (0,50,100)
2. Power Averaging **amplifies** the distance between probabilities.
3. This makes the order of ranks clearer = better AUC.

In case you want more explanation on Power Averaging, check out my previous notebooks:
* [Simple Power Averaging](https://www.kaggle.com/edrickkesuma/power-averaging-is-your-friend)
* [In depth Power Averaging](https://www.kaggle.com/edrickkesuma/in-depth-power-averaging-0-81848)

## What is VotingClassifier?

VotingClassifier takes a group of classifiers/models and **averages** out their predictions.

Remember that we need **highly correlated** submissions to get a good AUC. 

The easiest way to get them is to just change the **random_state** when creating each model. This way, the model makes predictions differently but they are still close to each other = high correlation.

You create several of these models with different random states and put them in VotingClassifier. 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datatable as dt

from xgboost import XGBClassifier
import time
from sklearn.ensemble import VotingClassifier

import gc

# Read in the data

In [2]:
%%time

train_data = dt.fread('../input/tabular-playground-series-oct-2021/train.csv')
test_data = dt.fread('../input/tabular-playground-series-oct-2021/test.csv')

print(train_data.shape, test_data.shape)

(1000000, 287) (500000, 286)
CPU times: user 13.3 s, sys: 2.99 s, total: 16.3 s
Wall time: 32.1 s


In [3]:
%%time 

# Credit: https://www.kaggle.com/hardyxu52/tps-oct-2021-reduce-memory-usage-but-faster
for i, col in enumerate(train_data):
    if col.type.name == 'float64':
        train_data[:,i] = dt.as_type(col, 'float32')

for i, col in enumerate(test_data):
    if col.type.name == 'float64':
        test_data[:,i] = dt.as_type(col, 'float32')

train_data = train_data.to_pandas()
test_data = test_data.to_pandas()

CPU times: user 5.19 s, sys: 1.25 s, total: 6.44 s
Wall time: 3.79 s


In [4]:
train_data = train_data.set_index('id', drop=True)
test_data = test_data.set_index('id', drop=True)
train_data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
id,,,,,,,,,,,,,,,,,,,,,
0,0.205979,0.410993,0.176775,0.223581,0.423543,0.476140,0.413590,0.612021,0.534873,0.147295,...,False,True,False,False,False,False,False,False,False,True
1,0.181004,0.473119,0.011734,0.213657,0.619678,0.441593,0.230407,0.686013,0.281971,0.238509,...,False,True,False,False,False,False,False,False,False,True
2,0.182583,0.307431,0.325950,0.207116,0.605699,0.309695,0.493337,0.751107,0.536272,0.286813,...,False,False,False,True,True,False,False,False,False,True
3,0.180240,0.494592,0.008367,0.223580,0.760618,0.439211,0.432055,0.776147,0.483958,0.260886,...,False,False,False,False,True,False,False,False,False,True
4,0.177172,0.495513,0.014263,0.548819,0.625396,0.562493,0.117158,0.561255,0.077115,0.158321,...,False,True,True,False,True,False,False,True,False,True


# Preprocessing

In [5]:
# Turn True/False into 0s and 1s
bool_cols_train = []
for i, col in enumerate(train_data.columns):
    if train_data[col].dtypes == bool:
        bool_cols_train.append(i)

In [6]:
bool_cols_test = []
for i, col in enumerate(test_data.columns):
    if test_data[col].dtypes == bool:
        bool_cols_test.append(i)

In [7]:
train_data.iloc[:, bool_cols_train] = train_data.iloc[:, bool_cols_train].astype(int)
test_data.iloc[:, bool_cols_test] = test_data.iloc[:, bool_cols_test].astype(int)

In [8]:
features = train_data.drop('target', axis=1).columns
label = 'target'

In [9]:
X = train_data[features].copy()
y = train_data[label].copy()
X_test = test_data.copy()

del train_data, test_data
gc.collect()

42

In [10]:
X['std'] = X.std(axis=1)
X['min'] = X.min(axis=1)
X['max'] = X.max(axis=1)
X['var'] = X.var(axis=1)

X_test['std'] = X_test.std(axis=1)
X_test['min'] = X_test.min(axis=1)
X_test['max'] = X_test.max(axis=1)
X_test['var'] = X_test.var(axis=1)

# Create models + VotingClassifier

In an ideal situation, you could run each batch on a different kernel for for **parallel** training and predicting.

In [11]:
# Credit: https://www.kaggle.com/shenurisumanasekara/tabular-october-xgbclassifier-stepbystep
# Set optimal hyperparameters
params = {
    'max_depth': 6,
    'n_estimators': 5500,
    'subsample': 0.6000000000000001,
    'colsample_bytree': 0.2,
    'colsample_bylevel': 0.4,
    'min_child_weight': 0.0475667709098205,
    'reg_lambda': 50.33144833870577,
    'reg_alpha': 0.01634917276171278,
    'gamma': 5.507875585868313,
    'booster': 'gbtree',
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'use_label_encoder': False
}

In [12]:
# Batch 1

xgb_clf1 = XGBClassifier(**params, random_state=1)
xgb_clf2 = XGBClassifier(**params, random_state=2)
xgb_clf3 = XGBClassifier(**params, random_state=3)
xgb_clf4 = XGBClassifier(**params, random_state=4)
xgb_clf5 = XGBClassifier(**params, random_state=5)

In [13]:
estimators=[('xgb1', xgb_clf1), 
            ('xgb2', xgb_clf2), 
            ('xgb3', xgb_clf3), 
            ('xgb4', xgb_clf4),
            ('xgb5', xgb_clf5)
           ]

start = time.time()
print(f'fitting ...')
model = VotingClassifier(estimators=estimators, voting='soft', verbose=True)
model.fit(X, y)

print('predicting ...')
model_preds1 = model.predict_proba(X_test)[:, -1]

elapsed = time.time() - start
print(f'elapsed time: {elapsed:.2f}sec\n')
print('model_preds1 ready!')

fitting ...
[Voting] ..................... (1 of 5) Processing xgb1, total= 3.8min
[Voting] ..................... (2 of 5) Processing xgb2, total= 3.7min
[Voting] ..................... (3 of 5) Processing xgb3, total= 3.8min
[Voting] ..................... (4 of 5) Processing xgb4, total= 3.8min
[Voting] ..................... (5 of 5) Processing xgb5, total= 3.7min
predicting ...
elapsed time: 1149.83sec

model_preds1 ready!


In [14]:
# Batch 2

xgb_clf6 = XGBClassifier(**params, random_state=6)
xgb_clf7 = XGBClassifier(**params, random_state=7)
xgb_clf8 = XGBClassifier(**params, random_state=8)
xgb_clf9 = XGBClassifier(**params, random_state=9)
xgb_clf10 = XGBClassifier(**params, random_state=10)

In [15]:
estimators=[('xgb6', xgb_clf6), 
            ('xgb7', xgb_clf7), 
            ('xgb8', xgb_clf8), 
            ('xgb9', xgb_clf9),
            ('xgb10', xgb_clf10)
           ]

start = time.time()
print(f'fitting ...')
model = VotingClassifier(estimators=estimators, voting='soft', verbose=True)
model.fit(X, y)

print('predicting ...')
model_preds2 = model.predict_proba(X_test)[:, -1]

elapsed = time.time() - start
print(f'elapsed time: {elapsed:.2f}sec\n')
print('model_preds2 ready!')

fitting ...
[Voting] ..................... (1 of 5) Processing xgb6, total= 3.8min
[Voting] ..................... (2 of 5) Processing xgb7, total= 3.7min
[Voting] ..................... (3 of 5) Processing xgb8, total= 3.7min
[Voting] ..................... (4 of 5) Processing xgb9, total= 3.7min
[Voting] .................... (5 of 5) Processing xgb10, total= 3.7min
predicting ...
elapsed time: 1142.93sec

model_preds2 ready!


In [16]:
# Batch 3

xgb_clf11 = XGBClassifier(**params, random_state=11)
xgb_clf12 = XGBClassifier(**params, random_state=12)
xgb_clf13 = XGBClassifier(**params, random_state=13)
xgb_clf14 = XGBClassifier(**params, random_state=14)
xgb_clf15 = XGBClassifier(**params, random_state=15)

In [17]:
estimators=[('xgb11', xgb_clf11), 
            ('xgb12', xgb_clf12), 
            ('xgb13', xgb_clf13), 
            ('xgb14', xgb_clf14),
            ('xgb15', xgb_clf15)
           ]

start = time.time()
print(f'fitting ...')
model = VotingClassifier(estimators=estimators, voting='soft', verbose=True)
model.fit(X, y)

print('predicting ...')
model_preds3 = model.predict_proba(X_test)[:, -1]

elapsed = time.time() - start
print(f'elapsed time: {elapsed:.2f}sec\n')
print('model_preds3 ready!')

fitting ...
[Voting] .................... (1 of 5) Processing xgb11, total= 3.8min
[Voting] .................... (2 of 5) Processing xgb12, total= 3.7min
[Voting] .................... (3 of 5) Processing xgb13, total= 3.7min
[Voting] .................... (4 of 5) Processing xgb14, total= 3.8min
[Voting] .................... (5 of 5) Processing xgb15, total= 3.7min
predicting ...
elapsed time: 1142.63sec

model_preds3 ready!


# Check for correlation

In [18]:
import matplotlib as plt
import plotly.figure_factory as ff
import plotly.express as px

group_labels = ['batch1', 'batch2', 'batch3']

data = np.corrcoef([model_preds1, model_preds2, model_preds3])
fig=px.imshow(data,x=group_labels, y=group_labels)

fig.show()

# Power averaging

The formula is **Final Predictions = (Predictions1^Power + Predictions2^Power + Predictions3^Power) / n_predictions**

In [19]:
sample_sub = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')
ensemble = sample_sub.copy()

power = 4

ensemble.loc[:,'target'] = (model_preds1**power + model_preds2**power + model_preds3**power)/3

In [20]:
ensemble

,id,target
0,1000000,0.345239
1,1000001,0.001235
2,1000002,0.645376
3,1000003,0.532305
4,1000004,0.010221
...,...,...
499995,1499995,0.897093
499996,1499996,0.484846
499997,1499997,0.006647
499998,1499998,0.115202


In [21]:
ensemble.to_csv('submission.csv', index=False)

# Closing thoughts

This strategy carried me to 20th place last TPS. Though, it seems that it doesn't work that well on this dataset. Particularly because the correlations aren't as high.. 

There might be potential for this to work if the model is trained differently. Eg. Changing learning rates and stacking from what I've seen.

Open to any suggestions.